<a href="https://colab.research.google.com/github/GKCSV/Resilia_ProjetoFinal_M6/blob/GK/Analise_de_despesas_publicas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**ANÁLISE DE DESPESAS PÚBLICAS GERADAS POR DEPUTADOS NOS ANOS DE 2019, 2020 E 2021**

#**1. Preparação dos dados para consultas:**

##1.1. Inicialização do ambiente e importação dos dados:

In [57]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from matplotlib.ticker import StrMethodFormatter
from sklearn.preprocessing import MinMaxScaler

In [58]:
despesas2019 = pd.read_csv('https://raw.githubusercontent.com/GKCSV/Resilia_ProjetoFinal_M6/main/Ano-2019.csv', sep = ';')
despesas2020 = pd.read_csv('https://raw.githubusercontent.com/GKCSV/Resilia_ProjetoFinal_M6/main/Ano-2020.csv', sep = ';')
despesas2021 = pd.read_csv('https://raw.githubusercontent.com/GKCSV/Resilia_ProjetoFinal_M6/main/Ano-2021.csv', sep = ';')

##1.2. Formatação do dataset:

###1.2.1. Mesclagem dos DataFrames para um único:

In [59]:
despesas1920 = pd.merge(despesas2019, despesas2020, how='outer')
despesas_gerais = pd.merge(despesas1920, despesas2021, how='outer')

###1.2.2. Visualizando as colunas presentes:

In [60]:
despesas_gerais.columns.tolist()

['txNomeParlamentar',
 'cpf',
 'ideCadastro',
 'nuCarteiraParlamentar',
 'nuLegislatura',
 'sgUF',
 'sgPartido',
 'codLegislatura',
 'numSubCota',
 'txtDescricao',
 'numEspecificacaoSubCota',
 'txtDescricaoEspecificacao',
 'txtFornecedor',
 'txtCNPJCPF',
 'txtNumero',
 'indTipoDocumento',
 'datEmissao',
 'vlrDocumento',
 'vlrGlosa',
 'vlrLiquido',
 'numMes',
 'numAno',
 'numParcela',
 'txtPassageiro',
 'txtTrecho',
 'numLote',
 'numRessarcimento',
 'vlrRestituicao',
 'nuDeputadoId',
 'ideDocumento',
 'urlDocumento']

###1.2.3 Exclusão de colunas não-utilizadas através do comando .drop():
###Serão removidas as colunas:

'cpf'

'nuCarteiraParlamentar'

'nuLegislatura'

'codLegislatura'

'numSubCota'

'numEspecificacaoSubCota'

'txtNumero'

'indTipoDocumento'

'numParcela'

'txtTrecho'

'numLote'

'numRessarcimento'

In [61]:
excluir_colunas = despesas_gerais.drop(columns=['cpf', 'nuCarteiraParlamentar', 'nuLegislatura', 'codLegislatura', 'numSubCota', 'numEspecificacaoSubCota', 'txtNumero', 'indTipoDocumento', 'numParcela', 'txtTrecho', 'numLote', 'numRessarcimento'])


###1.2.4. Criando cópia do dataset para atualização de nomeação:

In [62]:
despesas_selecionadas = excluir_colunas.copy()

###1.2.5 Visualizando os tipos de dados presentes nas colunas:

In [63]:
despesas_selecionadas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 555500 entries, 0 to 555499
Data columns (total 19 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   txNomeParlamentar          555500 non-null  object 
 1   ideCadastro                553759 non-null  float64
 2   sgUF                       553759 non-null  object 
 3   sgPartido                  553759 non-null  object 
 4   txtDescricao               555500 non-null  object 
 5   txtDescricaoEspecificacao  136177 non-null  object 
 6   txtFornecedor              555467 non-null  object 
 7   txtCNPJCPF                 531710 non-null  object 
 8   datEmissao                 539850 non-null  object 
 9   vlrDocumento               555500 non-null  float64
 10  vlrGlosa                   555500 non-null  float64
 11  vlrLiquido                 555500 non-null  float64
 12  numMes                     555500 non-null  int64  
 13  numAno                     55

###1.2.6. Corrigindo o formato da data:

In [64]:
despesas_selecionadas['datEmissao'] = pd.to_datetime(despesas_selecionadas['datEmissao'], format = '%Y/%m/%d')

###1.2.7. Exibindo a contagem de dados:

In [65]:
despesas_selecionadas.count()

txNomeParlamentar            555500
ideCadastro                  553759
sgUF                         553759
sgPartido                    553759
txtDescricao                 555500
txtDescricaoEspecificacao    136177
txtFornecedor                555467
txtCNPJCPF                   531710
datEmissao                   539850
vlrDocumento                 555500
vlrGlosa                     555500
vlrLiquido                   555500
numMes                       555500
numAno                       555500
txtPassageiro                129493
vlrRestituicao                  364
nuDeputadoId                 555500
ideDocumento                 555500
urlDocumento                 349639
dtype: int64

###1.2.8 Filtrando pelo Estado de MS:

In [66]:
despesas_ms = despesas_selecionadas.loc[despesas_selecionadas['sgUF']=='MS']

###1.2.9. Renomeando colunas:

In [67]:
despesas_ms.rename(columns={'txNomeParlamentar':'Nome-Parlamentar', 'ideCadastro':'ID-Parlamentar', 'sgUF':'Estado', 'sgPartido':'Partido', 'txtDescricao':'Despesa-Descrição', 'txtDescricaoEspecificacao':'Despesa-Especificação', 'txtFornecedor':'Nome-Fornecedor', 'txtCNPJCPF':'CNPJ/CPF-Fornecedor', 'datEmissao':'Emissão-NF', 'vlrDocumento':'Valor total-NF', 'vlrGlosa':'Glosa-NF', 'vlrLiquido':'Valor Liquido-NF', 'numMes':'Mês', 'numAno':'Ano', 'txtPassageiro':'Passageiro', 'vlrRestituicao':'Valor Restituição', 'nuDeputadoId':'ID-Parlamentar-Compra', 'ideDocumento':'Numero-NF', 'urlDocumento':'URL-NF'})

,Nome-Parlamentar,ID-Parlamentar,Estado,Partido,Despesa-Descrição,Despesa-Especificação,Nome-Fornecedor,CNPJ/CPF-Fornecedor,Emissão-NF,Valor total-NF,Glosa-NF,Valor Liquido-NF,Mês,Ano,Passageiro,Valor Restituição,ID-Parlamentar-Compra,Numero-NF,URL-NF
29077,Geraldo Resende,74374.0,MS,PSDB,TELEFONIA,NaN,RAMAL,000.000.000/0000-6,NaT,1.40,0.00,1.40,7,2019,NaN,NaN,1649,0,NaN
29078,Vander Loubet,74376.0,MS,PT,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,NaN,aguas guarirobas,040.895.700/0015-0,2019-01-14,77.42,0.00,77.42,1,2019,NaN,NaN,1697,6759317,https://www.camara.leg.br/cota-parlamentar/doc...
29079,Vander Loubet,74376.0,MS,PT,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,NaN,ENERGISA MS,154.138.260/0015-0,2019-01-11,257.93,7.95,249.98,1,2019,NaN,NaN,1697,6759322,https://www.camara.leg.br/cota-parlamentar/doc...
29080,Vander Loubet,74376.0,MS,PT,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,NaN,ronaldo guedine ribeiro,120.251.668/85 -,2019-02-07,2150.00,0.00,2150.00,1,2019,NaN,NaN,1697,6760118,https://www.camara.leg.br/cota-parlamentar/doc...
29081,Vander Loubet,74376.0,MS,PT,COMBUSTÍVEIS E LUBRIFICANTES.,Veículos Automotores,ALPHAVILE COMERCIO DE COMBUSTIVEIS LTDA,103.396.620/0014-4,2019-01-06,100.00,0.00,100.00,1,2019,NaN,NaN,1697,6759290,https://www.camara.leg.br/cota-parlamentar/doc...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553674,Bia Cavassa,207176.0,MS,PSDB,PASSAGEM AÉREA - RPA,NaN,Cia Aérea - TAM,020.128.620/0016-0,2021-02-04,1618.57,0.00,1618.57,2,2021,Bia Cavassa,NaN,3445,1745718,NaN
553675,Bia Cavassa,207176.0,MS,PSDB,PASSAGEM AÉREA - RPA,NaN,Cia Aérea - TAM,020.128.620/0016-0,2021-02-05,1085.47,0.00,1085.47,2,2021,Bia Cavassa,NaN,3445,1745744,NaN
553676,Bia Cavassa,207176.0,MS,PSDB,PASSAGEM AÉREA - RPA,NaN,Cia Aérea - TAM,020.128.620/0016-0,2021-02-10,2337.33,0.00,2337.33,2,2021,Bia Cavassa,NaN,3445,1745819,NaN
553677,Bia Cavassa,207176.0,MS,PSDB,PASSAGEM AÉREA - RPA,NaN,Cia Aérea - TAM,020.128.620/0016-0,2021-04-30,131.47,0.00,131.47,4,2021,Bia Cavassa,NaN,3445,1749934,NaN


###1.2.10. Finalização do DataFrame para utilização nas consultas:

In [68]:
despesas_publicas = despesas_ms.copy()
despesas_publicas

,txNomeParlamentar,ideCadastro,sgUF,sgPartido,txtDescricao,txtDescricaoEspecificacao,txtFornecedor,txtCNPJCPF,datEmissao,vlrDocumento,vlrGlosa,vlrLiquido,numMes,numAno,txtPassageiro,vlrRestituicao,nuDeputadoId,ideDocumento,urlDocumento
29077,Geraldo Resende,74374.0,MS,PSDB,TELEFONIA,NaN,RAMAL,000.000.000/0000-6,NaT,1.40,0.00,1.40,7,2019,NaN,NaN,1649,0,NaN
29078,Vander Loubet,74376.0,MS,PT,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,NaN,aguas guarirobas,040.895.700/0015-0,2019-01-14,77.42,0.00,77.42,1,2019,NaN,NaN,1697,6759317,https://www.camara.leg.br/cota-parlamentar/doc...
29079,Vander Loubet,74376.0,MS,PT,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,NaN,ENERGISA MS,154.138.260/0015-0,2019-01-11,257.93,7.95,249.98,1,2019,NaN,NaN,1697,6759322,https://www.camara.leg.br/cota-parlamentar/doc...
29080,Vander Loubet,74376.0,MS,PT,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,NaN,ronaldo guedine ribeiro,120.251.668/85 -,2019-02-07,2150.00,0.00,2150.00,1,2019,NaN,NaN,1697,6760118,https://www.camara.leg.br/cota-parlamentar/doc...
29081,Vander Loubet,74376.0,MS,PT,COMBUSTÍVEIS E LUBRIFICANTES.,Veículos Automotores,ALPHAVILE COMERCIO DE COMBUSTIVEIS LTDA,103.396.620/0014-4,2019-01-06,100.00,0.00,100.00,1,2019,NaN,NaN,1697,6759290,https://www.camara.leg.br/cota-parlamentar/doc...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553674,Bia Cavassa,207176.0,MS,PSDB,PASSAGEM AÉREA - RPA,NaN,Cia Aérea - TAM,020.128.620/0016-0,2021-02-04,1618.57,0.00,1618.57,2,2021,Bia Cavassa,NaN,3445,1745718,NaN
553675,Bia Cavassa,207176.0,MS,PSDB,PASSAGEM AÉREA - RPA,NaN,Cia Aérea - TAM,020.128.620/0016-0,2021-02-05,1085.47,0.00,1085.47,2,2021,Bia Cavassa,NaN,3445,1745744,NaN
553676,Bia Cavassa,207176.0,MS,PSDB,PASSAGEM AÉREA - RPA,NaN,Cia Aérea - TAM,020.128.620/0016-0,2021-02-10,2337.33,0.00,2337.33,2,2021,Bia Cavassa,NaN,3445,1745819,NaN
553677,Bia Cavassa,207176.0,MS,PSDB,PASSAGEM AÉREA - RPA,NaN,Cia Aérea - TAM,020.128.620/0016-0,2021-04-30,131.47,0.00,131.47,4,2021,Bia Cavassa,NaN,3445,1749934,NaN


#**2. Exportando o DataFrame "depesas_publicas" para um arquivo .csv:**

In [71]:
from google.colab import files
despesas_publicas.to_csv("despesas_publicas_ms" + '-subdirs_from_sitemap.csv', sep='\t')
files.download("despesas_publicas_ms" + '-subdirs_from_sitemap.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#**3. Análise dos dados:**

###2.1. - Qual o maior valor que um parlamentar gastou com um único fornecedor? Este fornecedor em específico possui alguma ligação com o parlamentar identificado?

###2.2. - Qual o maior valor gasto em uma única despesa de alimentação? Existe alguma explicação plausível?

###2.3. - Qual o percentual de notas fiscais indisponíveis para consulta? Nesta relação está inclusa alguma categoria que possa vir a configurar fraude?

###2.4 - De possíveis fraudes por falta de prestação de contas, qual o top 5 parlamentares?

###2.5 - Qual o parlamentar mais caro para o governo se considerado suas despesas pessoais e as despesas a partir de lideranças partidárias?

###2.6 - Existe alguma despesa duplicada entre parlamentar x liderança partidária?